In [ ]:
def partition(df,df_features,vo):
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(
                                        df[df_features], # datos
                                        df[vo],   # variable obj
                                        test_size=0.40, # partición
                                        random_state=1, # semilla
                                        shuffle=True) # elegir aleatoriamente
    

    
    return X_train, X_test, y_train, y_test
    

In [ ]:
def model_nn(X_train, X_test, y_train, random_state, activation):

        
    from sklearn.neural_network import MLPRegressor

    regr = MLPRegressor(max_iter=500000, activation=activation,random_state=random_state).fit(X_train, y_train)

    y_predict = regr.predict(X_test)
  


    return y_predict, regr


In [ ]:
def resultados(df):

    dfcom1 = y_test.reset_index()
    dfcom1_1 = df[['personal_id_x','customer_id','sbs_customer_id','taxpayer_id','segmento','gender_type']].reset_index()
    dfcom1_2 = dfcom1_1.merge(dfcom1, how='inner', on='index')

    dfcom2 = pd.concat([dfcom1_2, pd.DataFrame(y_predict, columns=["valor_pred"])],axis=1)

    dfcom2['error_est'] = dfcom2['vo_balance_amount']-dfcom2['valor_pred']
    dfcom2['p_error_est'] = abs(dfcom2['error_est']/dfcom2['vo_balance_amount'])

    dfcom3 = dfcom2['p_error_est'].sum()
    
    return dfcom2, dfcom3


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler  
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer

def ftransformacion(X_train, X_test, y_train, transf): 
    if transf!='ln':
        if transf=='minmax':
            scaler = MinMaxScaler()
            scaler2 = MinMaxScaler()
        elif transf=='standard':
            scaler = StandardScaler()
            scaler2 = StandardScaler()
        elif transf=='robust':
            scaler= RobustScaler()
            scaler2= RobustScaler()
        elif transf=='boxcox':
            scaler= PowerTransformer(method='yeo-johnson')
            scaler2= PowerTransformer(method='yeo-johnson')

        mm_scaler = scaler.fit(X_train)
        X_train = mm_scaler.transform(X_train)
        X_test = mm_scaler.transform(X_test)


        mm_scaler2 = scaler2.fit(pd.DataFrame(y_train))
        y_train = mm_scaler2.transform(y_train)
    else:
        X_train = np.log(X_train)
        X_test = np.log(X_test)
        #y_train = y_train.values
        y_train = np.log(y_train).values
        mm_scaler = ''
        mm_scaler2 = ''
        

    return X_train, X_test, y_train, mm_scaler, mm_scaler2
    


def transformacion_inversa(y_predict, mm_scaler2):
    if mm_scaler2!='':
        y_predict = mm_scaler2.inverse_transform(pd.DataFrame(y_predict))
    else: 
        y_predict = np.exp(y_predict)
        #y_predict = y_predict
    
    return y_predict